In [1]:
import os
import polars as pl
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import torch 
import torch.nn as nn
from typing import Optional, List, Union, Dict, Any
import kaggle_evaluation.jane_street_inference_server

In [2]:
# model1 = tf.keras.models.load_model("/kaggle/input/tfmodels/tensorflow2/default/1/tf_nn_model3.keras")
# model2 = tf.keras.models.load_model("/kaggle/input/tfmodels/tensorflow2/default/1/tf_nn_model4_no_concat.keras")

In [3]:
LSTM1 = torch.load('/kaggle/input/lstm-models-symbol-1-and-2/pytorch/default/1/LSTM_1.pth', weights_only= True)
LSTM2 = torch.load('/kaggle/input/lstm-models-symbol-1-and-2/pytorch/default/1/LSTM_2.pth', weights_only= True)

In [4]:

model_path = [
    "/kaggle/input/lstm-models-symbol-1-and-2/pytorch/default/1/LSTM_1.pth",
    "/kaggle/input/lstm-models-symbol-1-and-2/pytorch/default/1/LSTM_2.pth"
    
]

models = []
for pt in model_path:
    _model = torch.load(pt,weights_only=True)
    models.append(_model)

In [5]:
FEATS = [f"feature_{i:02d}" for i in range(79)]
TARGET = 'responder_6'
IX_IDS_BY_SYM = ['symbol_id','date_id','time_id']
sequence_length = 10  # Adjust as needed

## Functions to stack features by symbols

In [6]:
def check_cols(cols: List, df: pd.DataFrame) -> List:
    cols_valid = []
    for col in cols:
        if col in df.columns:
            cols_valid.append(col)
    return cols_valid

def stack_features_by_sym(data_all: pd.DataFrame,
                          feature_names: Optional[List] = None):
    if feature_names is None:
        feature_names = FEATS
    cols = IX_IDS_BY_SYM + ['weight'] + feature_names + [TARGET]
    cols = list(set(cols))
    cols = check_cols(cols, data_all)
    data_by_sym = data_all[cols]
    data_by_sym.set_index(IX_IDS_BY_SYM, append=True, drop=True,
                   inplace=True)
    data_by_sym = data_by_sym.droplevel(0, axis='index')
    data_by_sym = data_by_sym.unstack(level=['symbol_id'])
    data_by_sym = data_by_sym.ffill().fillna(0)

    #if isinstance(data_by_sym, pd.Series):
        #data_by_sym = data_by_sym.to_frame()

    return data_by_sym






In [7]:
'''
test_partition_9 = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9')
print(test_partition_9)
'''

"\ntest_partition_9 = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9')\nprint(test_partition_9)\n"

In [8]:
'''
test_partition_9_pd = test_partition_9.to_pandas()


last10 = test_partition_9_pd.query("date_id == 1698")
'''

'\ntest_partition_9_pd = test_partition_9.to_pandas()\n\n\nlast10 = test_partition_9_pd.query("date_id == 1698")\n'

In [9]:
'''
feature_names = FEATS
cols = IX_IDS_BY_SYM + ['weight'] + feature_names + [TARGET]
cols = list(set(cols))
data_all = last10
cols = check_cols(cols, data_all)
data_by_sym = data_all[cols]
data_by_sym.set_index(IX_IDS_BY_SYM, append=True, drop=True,
               inplace=True)
data_by_sym = data_by_sym.droplevel(0, axis='index')
data_by_sym = data_by_sym.unstack(level=['symbol_id'])
data_by_sym = data_by_sym.ffill().fillna(0)
'''

"\nfeature_names = FEATS\ncols = IX_IDS_BY_SYM + ['weight'] + feature_names + [TARGET]\ncols = list(set(cols))\ndata_all = last10\ncols = check_cols(cols, data_all)\ndata_by_sym = data_all[cols]\ndata_by_sym.set_index(IX_IDS_BY_SYM, append=True, drop=True,\n               inplace=True)\ndata_by_sym = data_by_sym.droplevel(0, axis='index')\ndata_by_sym = data_by_sym.unstack(level=['symbol_id'])\ndata_by_sym = data_by_sym.ffill().fillna(0)\n"

In [11]:
#data_by_sym

In [12]:
#last_10_stacked = stack_features_by_sym(last10)

In [13]:
#last_10_stacked

In [14]:
global last_rows_stacked

In [23]:
test_partition_9 = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9')
test_partition_9_pd = test_partition_9.to_pandas()
last10 = test_partition_9_pd.query("date_id == 1698")



In [20]:
last10

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
6236824,1698,0,0,3.743546,2.309442,1.404777,2.052407,1.860581,0.004939,0.188701,...,0.010417,-0.442215,-0.322407,0.143594,-0.926890,-0.782236,-0.036595,-1.305746,-0.795677,-0.143724
6236825,1698,0,1,2.826357,2.046231,1.634353,1.329780,1.441716,-0.201240,0.270867,...,-0.130599,-0.651829,-1.707840,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
6236826,1698,0,2,3.120292,1.885471,1.183647,1.501341,1.861803,-0.034465,0.476829,...,-0.116166,-0.656373,-0.264575,-0.892879,-1.511886,-1.033480,-0.378265,-1.574290,-1.863071,-0.027343
6236827,1698,0,3,2.638314,1.946201,0.473077,1.512463,2.250226,-0.000911,0.255018,...,-0.182941,-0.188186,-0.190970,-0.701490,0.098453,-1.015506,-0.054984,0.329152,-0.965471,0.576635
6236828,1698,0,4,1.852919,1.502017,0.926709,2.511769,2.168552,-0.224433,0.098944,...,0.117388,-0.257462,-0.471325,-0.297420,0.074018,-0.324194,-0.597093,0.219856,-0.276356,-0.904790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274571,1698,967,34,3.242493,2.525160,-0.721981,2.544025,2.477615,0.417557,0.785812,...,0.016936,0.243475,0.166927,0.384940,-0.174297,-0.066046,-0.038767,-0.132337,-0.022426,-0.252461
6274572,1698,967,35,1.079139,1.857906,-0.790646,2.745439,2.339877,0.845065,0.651370,...,0.050860,0.850152,0.909382,1.015314,0.235962,0.122539,0.099559,-0.249584,-0.123571,-0.460630
6274573,1698,967,36,1.033172,2.515527,-0.672298,2.289250,2.521592,0.255077,0.919892,...,0.152333,0.395684,-0.292574,-3.215846,-0.535129,-0.178484,-1.808150,-0.065355,-0.000367,-0.125170
6274574,1698,967,37,1.243116,2.663298,-0.889112,2.313155,3.101428,0.324454,0.618944,...,-0.029483,1.925987,0.479394,3.621867,-0.107114,-0.063599,1.204755,-0.148711,-0.026583,-0.256395


In [24]:
last_rows_stacked = stack_features_by_sym(last10)

last_rows_stacked = last_rows_stacked.iloc[-sequence_length:, :]



In [26]:
test_df = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet')



In [31]:
test_df = test_df.to_pandas()
test_df_stacked =  stack_features_by_sym(test_df)

In [32]:
test_df_stacked

feature_12                                               ...  \
symbol_id               0    1    2    3    4    5    6    7    8    9   ...   
date_id time_id                                                          ...   
0       0             -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0  ...   

                feature_64                                               
symbol_id               29   30   31   32   33   34   35   36   37   38  
date_id time_id                                                          
0       0             -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0  

[1 rows x 3120 columns]

In [33]:
df_by_sym_test = pd.concat((last_rows_stacked, test_df_stacked), axis=0).fillna(0.0)

In [34]:
df_by_sym_test

feature_12                                                    \
symbol_id               0         1         2         3         4         5    
date_id time_id                                                                
1698    958       0.488278  0.350213  0.303159  0.819930 -0.119557  0.178099   
        959       0.366581  0.340443  0.248542  0.907249 -0.144698  0.208104   
        960       0.281778  0.311317  0.237561  0.821619 -0.130338  0.228425   
        961       0.291007  0.352480  0.247245  0.733354 -0.176417  0.158833   
        962       0.281854  0.316873  0.366177  0.781481 -0.084440  0.289484   
        963       0.336648  0.259246  0.315678  0.861209 -0.146583  0.192232   
        964       0.458838  0.470127  0.285914  0.714540 -0.065786  0.336946   
        965       0.525306  0.407331  0.470415  0.891926 -0.097478  0.341964   
        966       0.354730  0.379341  0.349180  0.736268 -0.105363  0.225503   
        967       0.422208  0.370844  0.168929  0.755318 -0.072712  0.222310   
0       0        -0.000000 -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   

                                                         ... feature_64  \
symbol_id              6         7         8         9   ...         29   
date_id time_id                                          ...              
1698    958      0.150153  0.788217  0.483869  0.229291  ...  -0.450952   
        959      0.150562  1.089376  0.472412  0.188461  ...  -0.303638   
        960      0.176846  0.620961  0.503028  0.089498  ...  -0.336243   
        961      0.186456  0.593372  0.566578  0.171300  ...  -0.345467   
        962      0.150495  0.780520  0.416255  0.192972  ...  -0.269281   
        963      0.100175  0.708398  0.337724  0.199051  ...  -0.427742   
        964      0.186165  1.157851  0.566717  0.203002  ...  -0.394729   
        965      0.143095  0.715097  0.570875  0.174461  ...  -0.462115   
        966      0.196395  0.638356  0.448815  0.311392  ...  -0.374896   
        967      0.315203  1.048732  0.795884  0.266432  ...  -0.371880   
0       0       -0.000000 -0.000000 -0.000000 -0.000000  ...  -0.000000   

                                                                             \
symbol_id              30        31        32        33        34        35   
date_id time_id                                                               
1698    958     -0.422906 -0.299138 -0.287223 -0.333196 -0.475235 -0.305657   
        959     -0.345163 -0.327305 -0.135287 -0.497376 -0.400077 -0.183511   
        960     -0.296470 -0.353012 -0.374794 -0.261512 -0.495027 -0.408955   
        961     -0.424773 -0.330215 -0.435711 -0.346794 -0.535248 -0.310786   
        962     -0.628679 -0.438174 -0.162087 -0.297728 -0.435542 -0.326302   
        963     -0.473513 -0.314340 -0.378405 -0.317073 -0.451139 -0.406453   
        964     -0.348017 -0.256557 -0.360854 -0.333594 -0.261724 -0.433799   
        965     -0.420790 -0.354642 -0.412644 -0.440772 -0.333365 -0.213737   
        966     -0.356173 -0.304112 -0.353307 -0.353133 -0.371450 -0.234561   
        967     -0.304775 -0.354202 -0.438358 -0.356637 -0.408040 -0.278832   
0       0       -0.000000 -0.000000 -0.000000 -0.000000 -0.000000 -0.000000   

                                               
symbol_id              36        37        38  
date_id time_id                                
1698    958     -0.419962 -0.394540 -0.515163  
        959     -0.289614 -0.382479 -0.361249  
        960     -0.445389 -0.339621 -0.370870  
        961     -0.323119 -0.394677 -0.483410  
        962     -0.496259 -0.381747 -0.477698  
        963     -0.348060 -0.252270 -0.349789  
        964     -0.393790 -0.436228 -0.382677  
        965     -0.355145 -0.320026 -0.278163  
        966     -0.328170 -0.327631 -0.374481  
        967     -0.343650 -0.255192 -0.395652  
0       0       -0.000000 -0.000000 -0.000000  

[11 rows x 3159 columns]

## Test to try to make the seq length work on the test set (doesn't work atm)

In [ ]:
'''
# Define constants first
FEATS = [f"feature_{i:02d}" for i in range(79)]
TARGET = 'responder_6'
IX_IDS_BY_SYM = ['symbol_id']
sequence_length = 10  # Adjust as needed

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
   global list_test, lags_, model, cnt, cnt_day
   
   test_df = test.to_pandas().fillna(0.0)  
    
   feature_names = [f"feature_{i:02d}" for i in range(79)]
   
   # Get sequences for prediction
   X_test = test_stacked[feature_names]
   X_test_swap = X_test.swaplevel(axis=1)
   symbols = X_test_swap.columns.droplevel(1).unique()
   
   predictions = []
   
   with torch.no_grad():
       for s in symbols:
           if s in test_df['symbol_id'].values:
               X_sym = X_test_swap[s].values
               X_sequences = torch.tensor(X_sym, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
               
               pred = 0
               for model in models:
                   model.eval()
                   pred += model(X_sequences).squeeze().item() / len(models)
               
               predictions.append({
                   'row_id': test_df[test_df['symbol_id'] == s]['row_id'].iloc[0],
                   'responder_6': np.clip(pred, a_min=-5, a_max=5)
               })
   
   predictions_df = pd.DataFrame(predictions)
   return pl.from_pandas(predictions_df)
   '''

In [ ]:
global list_test, cnt, cnt_day
list_test = []
cnt_day = 0

Data Loading 

In [ ]:
test_partition_9 = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9')
print(test_partition_9)


test_df_stacked =  stack_features_by_sym(test_df)
df_by_sym_test = pd.concat((last_rows_stacked, test_df_stacked), axis=0).fillna(0.0)

In [51]:
input_size = 79
hidden_size = 64
num_layers = 2
num_epochs = 100
batch_size = 32
learning_rate = 0.001

class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, 
                           hidden_size=hidden_size, 
                           num_layers=num_layers, 
                           batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # Changed from linear to fc
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

'''
# Initialize models
models_lstm = {}
for checkpoint in [LSTM1, LSTM2]:
    model = LSTMModel()
    model.load_state_dict(checkpoint['model_state_dict'])  # Extract just the model state
    model.eval()
    models.append(model)
'''


# Initialiser le dictionnaire pour stocker les checkpoints
LSTM_chkpt = {}

# Charger les checkpoints
for s in [1, 2]:
    LSTM_chkpt[s] = torch.load(f'/kaggle/input/models-v2/pytorch/default/1/LSTM_{s}.pth', weights_only=True)  # Remplacez 'path_to_checkpoint' par le chemin correct

# Initialiser les modèles
models_lstm = {}
for s, checkpoint in LSTM_chkpt.items():
    model = LSTMModel()
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    models_lstm[s] = model 

# Define constants first
FEATS = [f"feature_{i:02d}" for i in range(79)]
TARGET = 'responder_6'
ix_ids_by_sym = ['symbol_id']
sequence_length = 10  # Adjust as needed

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
   global list_test, lags_, model, cnt, cnt_day
   
   test_df = test.to_pandas().fillna(0.0)
   test_df_stacked =  stack_features_by_sym(test_df)
   df_by_sym_test = pd.concat((last_rows_stacked, test_df_stacked), axis=0).fillna(0.0)
   #test_stacked = stack_features_by_sym(test_df)
   #test_stacked=test_stacked.to_frame() 
   #test_stacked=test_stacked.T 
   feature_names = FEATS
   
   # Get sequences for prediction
   X_test = df_by_sym_test[feature_names] 
   X_test_swap = X_test.swaplevel(axis=1)
   symbols = X_test_swap.columns.droplevel(1).unique()
   predictions = [] 
   with torch.no_grad():
       for s in symbols:
           if s in test_df['symbol_id'].values:
               X_sym = X_test_swap[s].values
               X_sequences = torch.tensor(X_sym, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
               model = models_lstm[int(s)]
               pred = model(X_sequences).squeeze().item()  

               '''
               for model in models:
                   model.eval()
                   pred += model(X_sequences).squeeze().item() / len(models)
               '''
               
               predictions.append({
                   'row_id': test_df[test_df['symbol_id'] == s]['row_id'].iloc[0],
                   'responder_6': np.clip(pred, a_min=-5, a_max=5)
               })
               

    


   
   predictions_df = pl.DataFrame(predictions)
   predictions_df = predictions_df.fill_null(0.0) 
   return predictions_df



In [52]:
%%time
# Setup the inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Running the inference server
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway((
        '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
        '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
    ))

CPU times: user 414 ms, sys: 319 ms, total: 733 ms
Wall time: 220 ms
